### 1. Recherche Full-Text avec tsvector et tsquery
PostgreSQL propose une puissante recherche plein-texte via les types tsvector et tsquery.

#### 📌 Comment ça marche ?
- ```tsvector``` : Représente un document sous une forme optimisée pour la recherche.
- ```tsquery``` : Représente une requête de recherche.
#### 🔹 Exemple de base :

In [ ]:
SELECT to_tsvector('french', 'Le chat noir dort sur le tapis')
       @@ to_tsquery('french', 'chat & tapis');

#### Explication :

- ```to_tsvector('french', 'Le chat noir dort sur le tapis')``` transforme le texte en une liste de tokens indexés.
- ```to_tsquery('french', 'chat & tapis')``` crée une requête qui cherche les mots "chat" et "tapis".
- ```@@``` vérifie si la requête correspond au texte.  

✅ Retourne ```true``` car les deux mots sont présents.

### 2️. Indexation et Optimisation avec GIN
#### 📌 Pourquoi utiliser un index ?
Sans index, chaque recherche nécessite un scan complet des lignes de la table. PostgreSQL propose GIN (Generalized Inverted Index) pour optimiser la recherche plein-texte.

- 🔹 Création d’un index full-text sur une colonne :

In [ ]:
CREATE INDEX idx_text_search ON articles USING GIN(to_tsvector('french', contenu));

- 🔹 Requête optimisée avec index :

In [ ]:
SELECT * FROM articles WHERE to_tsvector('french', contenu) @@ to_tsquery('french', 'chat & tapis');

✅ Gain de performance significatif !

### 3️. Dictionnaires Linguistiques
PostgreSQL propose plusieurs dictionnaires intégrés :

- simple : Sépare les mots mais ne fait pas de lemmatisation.
- english, french : Suppression des stopwords, lemmatisation ("mange" et "manger" deviennent équivalents).  


🔹 Test d’un dictionnaire :

In [ ]:
SELECT to_tsvector('french', 'Les étudiants apprennent la programmation');

In [ ]:
'étudiant':2 'apprendre':3 'programmation':4

✅ Les mots sont lemmatisés et "les" est supprimé (stopword).

### 4️. Recherche Approximative avec pg_trgm (Trigrammes)
La recherche full-text ne gère pas les fautes de frappe, mais l’extension pg_trgm permet d'effectuer des recherches approximatives.

- 📌 Installation de l’extension

In [ ]:
CREATE EXTENSION pg_trgm;

- 📌 Recherche approximative avec ```%```

In [ ]:
SELECT * FROM articles WHERE titre % 'programmtion';

✅ Trouve "programmation" même si "programmtion" contient une faute !

- 📌 Amélioration avec un index GIN

In [ ]:
CREATE INDEX idx_trigram ON articles USING GIN (titre gin_trgm_ops);

✅ Accélère considérablement la recherche approximative.

### 5️. Requêtes Avancées pour un Moteur de Recherche Performant
- 📌 Recherche mixte (full-text + trigrammes)

In [ ]:
SELECT * FROM articles 
WHERE to_tsvector('french', contenu) @@ to_tsquery('french', 'chat') 
ORDER BY similarity(contenu, 'chat') DESC;

✅ Trie les résultats selon leur similarité avec le mot recherché.

|Méthode|Avantages|Inconvénients|
|:---|:---|:---|
|```tsvector/tsquery```|Recherche rapide et précise|Ne gère pas les fautes|
|```pg_trgm```|Tolérance aux fautes|Plus lent sans index|
|```GIN```|Indexation efficace|Plus de stockage|